In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [2]:
data = data.drop(columns = ["ID","ZIP Code"])

In [3]:
X = data.drop("Personal Loan",axis=1)
y = data["Personal Loan"]

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [5]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
loss = 'binary_crossentropy'


C:\Users\Srivatsan\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [7]:
def fitness(weights):
    w0 = weights[:66].reshape((11, 6))
    w1 = weights[66:72].reshape((6,1))
    b0= np.array([0.,0.,0.,0.,0.,0.])
    b1 = np.array([0.])


    model.layers[0].set_weights([w0, b0])
    model.layers[1].set_weights([w1, b1])
    model.compile(optimizer=optimizer, loss=loss)
    Loss = model.evaluate(X_train, y_train, verbose=0)
    return -Loss

In [8]:
def genetic_algorithm(population_size, generations, mutation_rate):
    # Initialize the population and belief space with random weights
    population = [np.random.uniform(low=-1, high=1, size=77) for _ in range(population_size)]
    
    for generation in range(generations):
        fitness_scores = [fitness(x) for x in population]

        indices = np.argsort(fitness_scores)[-2:]
        parents = [population[i] for i in indices]

        offspring = []
        for _ in range(population_size - len(parents)):
            parent1 = parents[0].flatten()
            parent2 = parents[1].flatten()

            #Crossover
            crossover_point = np.random.randint(0, len(parent1))
            child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child = child.reshape(parents[0].shape)

            #Mutation
            for i in range(len(child)):
                if np.random.uniform() < mutation_rate:
                    child[i] += np.random.normal(loc=0, scale=0.1)

            offspring.append(child)

        # Combine parents and offspring to form the new population
        population = parents + offspring
        
        if (generation%5==0):
            print("Finished Generation:",generation)

    fitness_scores = [fitness(x) for x in population]
    best_index = np.argmax(fitness_scores)
    best_weights = population[best_index]

    return best_weights

In [9]:
model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

model.weights

[<tf.Variable 'dense/kernel:0' shape=(11, 6) dtype=float32, numpy=
 array([[-0.36340088,  0.5345434 , -0.3726328 , -0.01514661,  0.28678912,
          0.05968159],
        [-0.17813021,  0.258718  ,  0.579407  ,  0.07626575, -0.0099597 ,
         -0.03612757],
        [ 0.24457395,  0.03793025,  0.37907624,  0.4273678 , -0.02962226,
          0.3106345 ],
        [ 0.21546274,  0.14268339,  0.47586524, -0.17224076, -0.25019005,
          0.48467195],
        [-0.2759541 ,  0.17087376,  0.2913283 ,  0.55865204, -0.3290638 ,
         -0.14343381],
        [ 0.06859696,  0.07892281, -0.33100587, -0.45943484, -0.44637775,
         -0.28318435],
        [ 0.35193384,  0.36323005, -0.05805951, -0.48467016,  0.02380502,
         -0.41326684],
        [-0.11546889,  0.29461044,  0.31636947, -0.5789938 , -0.3031101 ,
          0.50401485],
        [ 0.04170316, -0.45354778, -0.44005245,  0.38928264,  0.5109402 ,
          0.55123854],
        [ 0.4157306 , -0.28336763,  0.0251798 ,  0.5098299 ,

In [10]:
best_weights = genetic_algorithm(population_size=20, generations=50, mutation_rate=0.1)
w0 = best_weights[:66].reshape((11, 6))
w1 = best_weights[66:72].reshape((6,1))
b0= np.array([0.,0.,0.,0.,0.,0.])
b1 = np.array([0.])

model.layers[0].set_weights([w0, b0])
model.layers[1].set_weights([w1, b1])

model.compile(optimizer=optimizer, loss=loss)

Finished Generation: 0
Finished Generation: 5
Finished Generation: 10
Finished Generation: 15
Finished Generation: 20
Finished Generation: 25
Finished Generation: 30
Finished Generation: 35
Finished Generation: 40
Finished Generation: 45


In [11]:
y_pred = model.predict(X_test)
y_pred = (y_pred < 0.00005)

47/47 [==============================] - 0s 1ms/step


In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8206666666666667